In [1]:
import pandas as pd
import numpy as np
import time

In [2]:
time.ctime()

'Mon May  4 19:31:13 2020'

In [3]:
# ２つのデータフレームの差が知りたい

In [4]:
df1 = pd.DataFrame(
    [
        ["00-000-01", 10, 0, 0, 0, 0],
        ["00-000-02", 0, 0, 0, 0, 0],
        ["00-000-03", 0, 0, 0, 0, 0],
        ["00-000-04", 10, 0, np.nan, np.nan, np.nan ],
        ["00-000-05", np.nan, np.nan, np.nan, np.nan, np.nan,]
    ],
    columns = ["ID", "VAL1", "VAL2", "VAL3", "VAL4", "VAL5",]
)
df2 = pd.DataFrame(
    [
        ["00-000-01", 10, 20, np.nan, np.nan, np.nan, ],
        ["00-000-03", 0, 10, 0, np.nan, np.nan],
        ["00-000-05", np.nan, np.nan, np.nan, np.nan, np.nan,],
        ["00-000-06", 10, 0, 0, 0, 0]
    ],
    columns = ["ID", "VAL1", "VAL2", "VAL3", "VAL4", "VAL5",]
)

In [5]:
%time
display(df1)
display(df2)

Wall time: 0 ns


,ID,VAL1,VAL2,VAL3,VAL4,VAL5
0,00-000-01,10.0,0.0,0.0,0.0,0.0
1,00-000-02,0.0,0.0,0.0,0.0,0.0
2,00-000-03,0.0,0.0,0.0,0.0,0.0
3,00-000-04,10.0,0.0,NaN,NaN,NaN
4,00-000-05,NaN,NaN,NaN,NaN,NaN


,ID,VAL1,VAL2,VAL3,VAL4,VAL5
0,00-000-01,10.0,20.0,NaN,NaN,NaN
1,00-000-03,0.0,10.0,0.0,NaN,NaN
2,00-000-05,NaN,NaN,NaN,NaN,NaN
3,00-000-06,10.0,0.0,0.0,0.0,0.0


In [6]:
# df1 と df2 に共通するものだけに絞る
commID = set(df1["ID"]) & set(df2["ID"])
commID

{'00-000-01', '00-000-03', '00-000-05'}

In [7]:
df1_com = df1[ df1["ID"].isin(commID) ]
df2_com = df2[ df2["ID"].isin(commID) ]

display( df1_com )
display( df2_com )

,ID,VAL1,VAL2,VAL3,VAL4,VAL5
0,00-000-01,10.0,0.0,0.0,0.0,0.0
2,00-000-03,0.0,0.0,0.0,0.0,0.0
4,00-000-05,NaN,NaN,NaN,NaN,NaN


,ID,VAL1,VAL2,VAL3,VAL4,VAL5
0,00-000-01,10.0,20.0,NaN,NaN,NaN
1,00-000-03,0.0,10.0,0.0,NaN,NaN
2,00-000-05,NaN,NaN,NaN,NaN,NaN


In [8]:
# df1_com == df2_com
# インデックスが違う(0,2,4 ⇔ 0,1,2)ので、比較ができない

In [9]:
# インデックスを振りなおしてやる
df1_com.reset_index(drop=True, inplace=True)
df2_com.reset_index(drop=True, inplace=True)
display( df1_com )
display( df2_com )

,ID,VAL1,VAL2,VAL3,VAL4,VAL5
0,00-000-01,10.0,0.0,0.0,0.0,0.0
1,00-000-03,0.0,0.0,0.0,0.0,0.0
2,00-000-05,NaN,NaN,NaN,NaN,NaN


,ID,VAL1,VAL2,VAL3,VAL4,VAL5
0,00-000-01,10.0,20.0,NaN,NaN,NaN
1,00-000-03,0.0,10.0,0.0,NaN,NaN
2,00-000-05,NaN,NaN,NaN,NaN,NaN


In [10]:
df1_com == df2_com
# Nan同士は比較できないので何かで埋めてやらないとダメ

,ID,VAL1,VAL2,VAL3,VAL4,VAL5
0,True,True,False,False,False,False
1,True,True,False,True,False,False
2,True,False,False,False,False,False


In [11]:
# 一致していないカラムをリスト化
# 今回ほんとにやりたいのは何万レコード*34列位だからここは手動でもいいや
tmpDiffSum = (df1_com == df2_com).all()
display(tmpDiffSum)
print("型:", type(tmpDiffSum))
print("index:", tmpDiffSum.index)

ID       True
VAL1    False
VAL2    False
VAL3    False
VAL4    False
VAL5    False
dtype: bool

型: <class 'pandas.core.series.Series'>
index: Index(['ID', 'VAL1', 'VAL2', 'VAL3', 'VAL4', 'VAL5'], dtype='object')


In [12]:
tmpUnmatchCol = []
for i in tmpDiffSum.index:
    if tmpDiffSum[i] == False:
        tmpUnmatchCol.append(i)
tmpUnmatchCol

['VAL1', 'VAL2', 'VAL3', 'VAL4', 'VAL5']

In [13]:
# 各DFで不一致だったカラム
tmpDf_UnmatchCol = pd.concat([df1_com[tmpUnmatchCol], df2_com[tmpUnmatchCol]], axis=0)
tmpDf_UnmatchCol

,VAL1,VAL2,VAL3,VAL4,VAL5
0,10.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0
2,NaN,NaN,NaN,NaN,NaN
0,10.0,20.0,NaN,NaN,NaN
1,0.0,10.0,0.0,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN


In [14]:
# 各DFで不一致だったカラムの値
for col in tmpDf_UnmatchCol.columns:
    print(col,tmpDf_UnmatchCol[col].unique())

VAL1 [10.  0. nan]
VAL2 [ 0. nan 20. 10.]
VAL3 [ 0. nan]
VAL4 [ 0. nan]
VAL5 [ 0. nan]


In [41]:
# もし登場する値で埋めてしまうと、不一致なのに、一致になってしまうので、
# 登場しない値で埋める
df1_com_fill = df1_com.copy()
df2_com_fill = df2_com.copy()

df1_com_fill["VAL1"].fillna(100, inplace = True)
df1_com_fill["VAL2"].fillna(100, inplace = True)
df1_com_fill["VAL3"].fillna(100, inplace = True)
df1_com_fill["VAL4"].fillna(100, inplace = True)
df1_com_fill["VAL5"].fillna(100, inplace = True)
df2_com_fill["VAL1"].fillna(100, inplace = True)
df2_com_fill["VAL2"].fillna(100, inplace = True)
df2_com_fill["VAL3"].fillna(100, inplace = True)
df2_com_fill["VAL4"].fillna(100, inplace = True)
df2_com_fill["VAL5"].fillna(100, inplace = True)

In [42]:
# nullがなくなったことのチェック
display( df1_com_fill.isnull().any() )
display( df2_com_fill.isnull().any() )

ID      False
VAL1    False
VAL2    False
VAL3    False
VAL4    False
VAL5    False
dtype: bool

ID      False
VAL1    False
VAL2    False
VAL3    False
VAL4    False
VAL5    False
dtype: bool

In [43]:
# 改めて比較
df1_com_fill == df2_com_fill

,ID,VAL1,VAL2,VAL3,VAL4,VAL5
0,True,True,False,False,False,False
1,True,True,False,True,False,False
2,True,True,True,True,True,True


In [44]:
col_match = (df1_com_fill == df2_com_fill).all()
col_match

ID       True
VAL1     True
VAL2    False
VAL3    False
VAL4    False
VAL5    False
dtype: bool

In [45]:
# all(axis=1)でレコード毎に一致するかどうかが出せる
row_match = (df1_com_fill == df2_com_fill).all(axis=1)
row_match

0    False
1    False
2     True
dtype: bool

In [46]:
# カラムでいうとVAL1以外、レコードでいうとindex=2以外は不一致ってのがわかった
# けど、具体的にどう違うのかってのが知りたい

In [47]:
# row_match=FALSEのインデックスを指定して、マッチしないレコードをとる
df1_un = df1_com_fill.iloc[ row_match[row_match==False].index]
df2_un = df2_com_fill.iloc[ row_match[row_match==False].index]
# 次にcancatして並び替えるので、df1,df2どちらのレコードなのかわかるように色を付ける
df1_un["DATA"] = "DATA_1"
df2_un["DATA"] = "DATA_2"

df_un = pd.concat([df1_un, df2_un], axis=0)
df_un[["ID", "DATA", "VAL2","VAL3","VAL4","VAL5",]].sort_values(["ID", "DATA"])

C:\MyProgram\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
C:\MyProgram\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,ID,DATA,VAL2,VAL3,VAL4,VAL5
0,00-000-01,DATA_1,0.0,0.0,0.0,0.0
0,00-000-01,DATA_2,20.0,100.0,100.0,100.0
1,00-000-03,DATA_1,0.0,0.0,0.0,0.0
1,00-000-03,DATA_2,10.0,0.0,100.0,100.0


In [48]:
# 差異があるレコードが数十レコードであれば、↑をだせば、目視なり、エクセルに貼るなりですぐわかるんだろうけど。。
# 数百、数戦あったら？？

In [49]:
# 汎用的にも考えたいが、いまは目の前の問題を対処するために愚直なやり方でもいいや、

In [50]:
df_TF = df1_com_fill == df2_com_fill
df_TF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 6 columns):
ID      3 non-null bool
VAL1    3 non-null bool
VAL2    3 non-null bool
VAL3    3 non-null bool
VAL4    3 non-null bool
VAL5    3 non-null bool
dtypes: bool(6)
memory usage: 146.0 bytes


In [51]:
df_TFbk = df_TF.copy()
for col in df_TF.columns:
    df_TF[col] = df_TF[col].astype("object")
df_TF

,ID,VAL1,VAL2,VAL3,VAL4,VAL5
0,True,True,False,False,False,False
1,True,True,False,True,False,False
2,True,True,True,True,True,True


In [52]:

for col in df_TF.columns:
    if col != "ID":
        df_TF[col] = df_TF[col].mask(df_TF[col] == True, np.nan)
        df_TF[col] = df_TF[col].mask(df_TF[col] == False, 1)
    else:
        df_TF[col] = 1
df_TF

,ID,VAL1,VAL2,VAL3,VAL4,VAL5
0,1,NaN,1,1,1,1
1,1,NaN,1,NaN,1,1
2,1,NaN,NaN,NaN,NaN,NaN


In [53]:
display(df1_com_fill * df_TF)
display(df2_com_fill * df_TF)
# こうすると、ちょっとは見やすくなるけども。

,ID,VAL1,VAL2,VAL3,VAL4,VAL5
0,00-000-01,NaN,0,0,0,0
1,00-000-03,NaN,0,NaN,0,0
2,00-000-05,NaN,NaN,NaN,NaN,NaN


,ID,VAL1,VAL2,VAL3,VAL4,VAL5
0,00-000-01,NaN,20,100,100,100
1,00-000-03,NaN,10,NaN,100,100
2,00-000-05,NaN,NaN,NaN,NaN,NaN


In [54]:
# まあいまは正の0～30くらいまでしか入ってない数字列だけがちがっているとしよう
# 仮に文字列が違うならそれこそれこそ比較って高度になるし、

In [64]:
# row_match=FALSEのインデックスを指定して、マッチしないレコードをとる
df1_un = df1_com_fill.iloc[ row_match[row_match==False].index]
df2_un = df2_com_fill.iloc[ row_match[row_match==False].index]

# マッチしないレコードについて、一致⇔不一致をとって、True/Falseではなく、Nan/１に変換
df_TF = df1_un == df2_un
for col in df_TF.columns:
    df_TF[col] = df_TF[col].astype("object")
for col in df_TF.columns:
    if col != "ID":
        df_TF[col] = df_TF[col].mask(df_TF[col] == True, np.nan)
        df_TF[col] = df_TF[col].mask(df_TF[col] == False, 1)
    else:
        df_TF[col] = 1

# マッチしないレコード　* Nan/１ をかけて、一致しないカラム以外はNanにする
df1_tmp = (df1_un * df_TF)[["VAL1","VAL2","VAL3","VAL4","VAL5"]]
df2_tmp = (df2_un * df_TF)[["VAL1","VAL2","VAL3","VAL4","VAL5"]]

# 引き算でどれだけ違うが計算
df1_tmp - df2_tmp

,VAL1,VAL2,VAL3,VAL4,VAL5
0,NaN,-20,-100,-100,-100
1,NaN,-10,NaN,-100,-100


In [69]:
# わかりにくいので、PKであるIDを追加
df_diff_final = pd.concat([df1_un[["ID"]], df1_tmp - df2_tmp], axis=1)
df_diff_final

,ID,VAL1,VAL2,VAL3,VAL4,VAL5
0,00-000-01,NaN,-20,-100,-100,-100
1,00-000-03,NaN,-10,NaN,-100,-100


In [71]:
# いちおうこれで、数値列については、
# df1 - df2 の結果が出せて、
# これを見るとすべてマイナスということがわかり、
#　（数値が上がっていくというストーリだが）
# df2のほうが大きいということがわかる
df_diff_final[["VAL1","VAL2","VAL3","VAL4","VAL5"]] > 0

,VAL1,VAL2,VAL3,VAL4,VAL5
0,False,False,False,False,False
1,False,False,False,False,False
